In [138]:
import pandas as pd
from os import path, curdir

ROOT = path.abspath(curdir)

datapath = path.join(ROOT, 'data', 'data.csv')
datapath2 = path.join(ROOT, 'data', 'calculations.csv')
#Join calculations table
df = pd.read_csv(datapath)
df2 = pd.read_csv(datapath2)
df = pd.merge(df, df2, how= 'left', on='State')
df = df.iloc[:,1:]
df = df.rename(columns={'RecidivismRate_x':'RecidivismRate'})
df=df.drop(columns=['Unnamed: 0_y','RecidivismRate_y'])
df['PercentViolentCrimes'] = df['TotalViolentCrimes']/(df['TotalViolentCrimes']+df['TotalNonviolentCrimes']+df['OtherCrimes'])
print(df.columns)



Index(['State', 'RecidivismRate', 'PrisonPopulation', 'ImprisonmentRate',
       'Population', 'qualityoflifetotalscore', 'QualityOfLifeAffordability',
       'QualityOfLifeEconomy', 'QualityOfLifeEducationAndHealth',
       'QualityOfLifeSafety',
       ...
       'PercentAsian', 'PercentWhite', 'PercentBlack', 'PercentNativeAmerican',
       'PercentOfPopInPrison', 'TotalViolentCrimes', 'TotalNonviolentCrimes',
       'OtherCrimes', 'AverageQOLScore', 'PercentViolentCrimes'],
      dtype='object', length=119)


In [146]:
features = ['PercentMale', 
            'PercentAsian',
            'PercentWhite',
            'PercentBlack',
            'PercentNativeAmerican',
            'AverageQOLScore',
            'PercentViolentCrimes',
            'PercentOfPopInPrison']
response = ['RecidivismRate']

In [170]:
from sklearn.model_selection import train_test_split
X = df[features]
Y = df[response]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, train_size=0.8)


In [ ]:
from sklearn import linear_model as lm
from sklearn.metrics import mean_squared_error, r2_score
for i in range(1, 2):
    model = lm.LinearRegression()
    model.fit(X_train, Y_train)
    score = model.score(X_test, Y_test)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(Y_test, y_pred)
    r2 = r2_score(Y_test, y_pred)
    print(f"Alpha: {0.1*i:.2f}; Score: {score:.2f}; MSE: {mse:.2f}; R2: {r2:.2f}")
variable_dict = {"colnames": [], "coefficients": []}
for i in range(len(features)):
    if model.coef_[0][i] != 0:
        variable_dict["colnames"].append(features[i])
        variable_dict["coefficients"].append(model.coef_[0][i])
coef_data_frame = pd.DataFrame.from_dict(variable_dict)
datapath3 = path.join(ROOT, 'data', 'coefficients.csv')
coef_data_frame.to_csv(datapath3)

predictions = model.predict(X)
data_dict = {'states Y': 'predictedRecidivismRate': predictions[0]}
predictDF = pd.DataFrame.from_dict(data_dict)

Alpha: 0.10; Score: -2.07; MSE: 169.89; R2: -2.07


In [142]:
from sklearn import tree
treeModel = tree.DecisionTreeRegressor()
treeModel.fit(X_train,Y_train)
treeModel.score(X_test,Y_test)

-0.964289885803107

In [ ]:
from numpy import ravel
y_train_1d = ravel(Y_train)
from sklearn.ensemble import RandomForestRegressor
treeModel = RandomForestRegressor(n_estimators=8)
treeModel.fit(X_train, y_train_1d)
score = treeModel.score(X_test,Y_test)
y_pred = treeModel.predict(X_test)
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)
print(f"Score: {score}; MSE: {mse}; R2: {r2}")




Score: 0.048297424324037275; MSE: 112.77276041666667; R2: 0.048297424324037275


In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

numeric_cols = df.select_dtypes(include=[np.number]).columns  # Select numeric columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean()) 

# Select relevant features
target = 'RecidivismRate'  # Define target variable
features = [col for col in df.columns if col != target and col != 'State']  # Drop 'State' if categorical
X = df[features]
y = df[target]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Select only numeric columns
df_numeric = df.select_dtypes(include=[np.number])

In [144]:
model = Lasso(alpha=0.5)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"r2: {r2}")
print(f"mse: {mse}")

included_vars = []
coefs = []
variable_dict = {"colnames": [], "coefficients": []}

for i in range(len(features)):
    if model.coef_[i] != 0:
        included_vars.append(features[i])
        coefs.append(model.coef_[i])
        variable_dict["colnames"].append(features[i])
        variable_dict["coefficients"].append(model.coef_[i])

print(variable_dict)

print(mse)
print(r2)

model.score(X_test, y_test)

coef_data_frame = pd.DataFrame.from_dict(variable_dict)
print(ROOT)
datapath3 = path.join(ROOT, 'data', 'coefficients.csv')
coef_data_frame.to_csv(datapath3)




r2: -0.00812548190243656
mse: 150.60075424300408
{'colnames': ['PercentBlack', 'PercentNativeAmerican', 'PercentOfPopInPrison'], 'coefficients': [np.float64(2.0917462770175397), np.float64(4.260640782106081), np.float64(0.5781508361011193)]}
150.60075424300408
-0.00812548190243656
c:\Users\joshu\School\DS-4010\CrimeFightersRepo\CrimeDashboard\modelling\workareas\Josh


In [32]:
print(features)

['RecidivismRate', 'PrisonPopulation', 'Population', 'qualityoflifetotalscore', 'QualityOfLifeAffordability', 'QualityOfLifeEconomy', 'QualityOfLifeEducationAndHealth', 'QualityOfLifeSafety', 'Male', 'Female', 'Total', 'Rape', 'Arson', 'Fraud', 'Larceny', 'Robbery', 'Runaway', 'Weapons', 'Burglary', 'Gambling', 'Vagrancy', 'Suspicion', 'Vandalism', 'Drunkenness', 'Embezzlement', 'Sex_Offenses', 'Rape_Legacy', 'Simple_Assault', 'Drug_Possession', 'Stolen_Property', 'Human_Trafficking', 'Aggravated_Assault', 'All_Other_Offenses', 'Disorderly_Conduct', 'Motor_Vehicle_Theft', 'Drug_Abuse_Violations', 'Liquor_Law_Violations', 'Counterfeiting_or_Forgery', 'Drug_Sale_or_Manufacturing', 'Drive_Under_the_Influence', 'Manslaughter_by_Negligence', 'Murder_and_Nonnegligent_Homicide', 'Curfew_and_Loitering_Law_Violations', 'Prostitution_and_Commercialized_Vice', 'Offenses_Against_the_Family_and_Children', 'Asian', 'White', 'Unknown', 'Multiple', 'NotSpecified', 'Black', 'NativeAmerican', 'NativeHaw